<b>This project focuses on the Fama French Three Factor Regression Model for Superconductor Technologies Inc.'s stock. The company was randomly picked. The project will be incorporating Numpy techniques and Statsmodels API. The company's stock returns were extracted from yahoo finance https://finance.yahoo.com/quote/SCON/ two years ago and the Fama French Factors were extracted from http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html two years ago, too.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
def float_formatter(x):
    return "{:7.4f}".format(x)

In [3]:
np.set_printoptions(formatter={'float_kind':float_formatter})

In [4]:
def reg_m(y, x):
    X = np.hstack((np.ones((len(x),1)), x))     # adds column of ones to X
    results = sm.OLS(y, X).fit()                # creates object containing regression results
    return results

<b>The code above is useful and will help facilitate the regression output. This code was created and shared by Prof. Jonathan Reuter, PhD

<b>Step 1:</b> Estimate the Fama-French Three-Factor Model for your stock.Use the regression output---and if-else statements---to determine (and print) whether your alpha is statistically significantly different from zero (at 5% level in a two-sided hypothesis test) and whether any of the estimated beta is statistically significantly different from zero (at 5% level in a two-sided hypothesis test).

In [5]:
doc = np.load('SCON-monthly.npy')
names = np.array(['yyyymmdd','year','close', 'mret', 'rf', 'mktrf', 'smb', 'hml']) 
print(doc.shape) 

(36, 8)


In [6]:
rf = doc[:,names=='rf'].reshape(36,1)
mktrf = doc[:,names=='mktrf'].reshape(36,1)
smb = doc[:,names=='smb'].reshape(36,1)
hml = doc[:,names=='hml'].reshape(36,1)
y = doc[:,names=='mret'].reshape(36,1) - rf 
X = np.hstack((mktrf,smb,hml))

In [7]:
reg_m(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.180
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     2.336
Date:                Fri, 11 Sep 2020   Prob (F-statistic):             0.0924
Time:                        02:35:24   Log-Likelihood:                 2.4903
No. Observations:                  36   AIC:                             3.019
Df Residuals:                      32   BIC:                             9.354
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9433      0.042     22.457      0.000       0.858       1.029
x1            -0.2311      1.388     -0.167      0.869      -3.058       2.596
x2             4.7094      1.811      2.600      0.014       1.020       8.399
x3            -0.8420      1.611     -0.523      0.605      -4.123       2.439
==============================================================================
Omnibus:                        3.081   Durbin-Watson:                   2.113
Prob(Omnibus):                  0.214   Jarque-Bera (JB):                2.072
Skew:                           0.194   Prob(JB):                        0.355
Kurtosis:                       4.110   Cond. No.                         50.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
result = reg_m(y, X)

In [9]:
alpha = result.params[0]
alpha = float_formatter(alpha)
a_pvalue = result.pvalues[0]
a_pvalue = float_formatter(a_pvalue)
print(f'The alpha is statistically significant from 0 with a coefficient of {alpha} and a p-value of {a_pvalue}')

The alpha is statistically significant from 0 with a coefficient of  0.9433 and a p-value of  0.0000


In [10]:
beta = result.params[1:] 
b_pvalues = result.pvalues[1:]
a1 = np.array([beta[0],b_pvalues[0]])
a2 = np.array([beta[1],b_pvalues[1]])
a3 = np.array([beta[2],b_pvalues[2]])
final_array = np.array([a1,a2,a3])
for i in final_array: 
    if i[1] < 0.05: 
        print(f'The beta {float_formatter(i[0])} is statistically significant from 0 with a p-value of {float_formatter(i[1])}')
    else: 
        print(f'The beta {float_formatter(i[0])} is not statistically significant from 0 with a p-value of {float_formatter(i[1])}')

The beta -0.2311 is not statistically significant from 0 with a p-value of  0.8688
The beta  4.7094 is statistically significant from 0 with a p-value of  0.0140
The beta -0.8420 is not statistically significant from 0 with a p-value of  0.6048


<b>Step 2:</b> You want to backtest a simple trading strategy that has you earning the monthly return of the stock from previous results in odd months (January, March, ...) and the monthly risk-free rate of return in the even months (February, April, ...). For simplicity, assume that you invest the same amount of money in this strategy every month, so that you can focus on average monthly returns rather than dollar-weighted average monthly returns. 

- Calculate (and print) the average monthly return of this trading strategy. 
- Estimate the CAPM alpha and beta associated with this trading strategy. Use the regression output---and if-else statements---to determine whether your CAPM alpha is statistically significantly different from zero (at 5% level in a two-sided hypothesis test) and whether the estimated CAPM beta is statistically significantly different from zero (at 5% level in a two-sided hypothesis test). 

In [11]:
n = np.load('SCON-monthly.npy')
n_names = np.array(['yyyymm', 'year', 'close', 'mret', 'rf', 'mktrf', 'smb', 'hml']) 
rf_even = []
mret_odd = []
for i in n: 
    if i[0]%2 == 0: 
        rf_even.append(i)
    else: 
        mret_odd.append(i)    

In [12]:
for ele in rf_even: 
    ele[3]=ele[4] 

In [13]:
list_all = rf_even + mret_odd
mret_odd_new = []
def takefirst(l):
    return l[0]

for i in range(len(mret_odd)):
    row = []
    for j in range(len(mret_odd[i])):
        row.append(mret_odd[i][j])
    mret_odd_new.append(row)
rf_even_new = []
for i in range(len(rf_even)):
    row = []
    for j in range(len(rf_even[i])):
        row.append(rf_even[i][j])
    rf_even_new.append(row)
list_all = mret_odd_new + rf_even_new
list_all.sort(key=takefirst)

In [14]:
array_all = np.array(list_all) 
a_names = np.array(['yyyymm', 'year', 'close', 'mret', 'rf', 'mktrf', 'smb', 'hml'])
mo_ret = array_all[:,a_names=='mret'] 
avg_monthly_return = np.mean(mo_ret)
print(float_formatter(avg_monthly_return))

 0.5063


In [15]:
rf = array_all[:, a_names=='rf'].reshape(36,1)
y = array_all[:, a_names=='mret'].reshape(36,1) - rf
mktrf = array_all[:, a_names=='mktrf'].reshape(36,1)

In [16]:
reg_m(y,mktrf).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.6993
Date:                Fri, 11 Sep 2020   Prob (F-statistic):              0.409
Time:                        02:35:29   Log-Likelihood:                -28.129
No. Observations:                  36   AIC:                             60.26
Df Residuals:                      34   BIC:                             63.42
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4820      0.095      5.070      0.000       0.289       0.675
x1             2.5708      3.074      0.836      0.409      -3.677       8.818
==============================================================================
Omnibus:                       15.907   Durbin-Watson:                   3.688
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                3.353
Skew:                           0.256   Prob(JB):                        0.187
Kurtosis:                       1.595   Cond. No.                         33.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
result2 = reg_m(y,mktrf)

In [18]:
alpha = result2.params[0]
alpha = float_formatter(alpha)
a_pvalue = result2.pvalues[0]
a_pvalue = float_formatter(a_pvalue)
print(f'The alpha is statistically significant from 0 with a coefficient of {alpha} and a p-value of {a_pvalue}')

The alpha is statistically significant from 0 with a coefficient of  0.4820 and a p-value of  0.0000


In [19]:
beta = result2.params[1]
beta = float_formatter(beta)
b_pvalues = result2.pvalues[1]
b_pvalues = float_formatter(b_pvalues)
print(f'The beta is not statistically significant from 0 with a coefficient of {beta} and a p-value of {b_pvalues}')

The beta is not statistically significant from 0 with a coefficient of  2.5708 and a p-value of  0.4089
